# Simeon Omeda: Augmenting Pandas with SQLite

## Question 1: Introduction

In this session, we explored a few different ways to work with larger datasets in pandas. In this guided project, we'll practice using some of the techniques we learned to analyze startup investments from Crunchbase.com.

Every year, thousands of startup companies raise financing from investors. Each time a startup raises money, we refer to the event as a fundraising round. Crunchbase is a website that crowdsources information on the fundraising rounds of many startups. The Crunchbase user community submits, edits, and maintains most of the information in Crunchbase.

In return, Crunchbase makes the data available through a Web application and a fee-based API. Before Crunchbase switched to the paid API model, multiple groups crawled the site and released the data online. Because the information on the startups and their fundraising rounds is always changing, the data set we'll be using isn't completely up to date.

Throughout this project, we'll practice working with different memory constraints. In this step, let's assume we only have 10 megabytes of available memory. While crunchbase-investments.csv (https://bit.ly/3BPcobU) consumes 10.3 megabytes of disk space, we know from earlier lessons that pandas often requires 4 to 6 times amount of space in memory as the file does on disk (especially when there's many string columns).


**Tasks**

* Because the data set contains over 50,000 rows, you'll need to read the data set into dataframes using 5,000 row chunks to ensure that each chunk consumes much less than 10 megabytes of memory.
* Across all of the chunks, become familiar with:
1. Each column's missing value counts.
2. Each column's memory footprint.
3. The total memory footprint of all of the chunks combined.
4. Which column(s) we can drop because they aren't useful for analysis.



In [10]:
# Your code goes here
# Dataset URL = https://bit.ly/3BPcobU
#
import pandas as pd

df = pd.read_csv("https://bit.ly/3BPcobU",encoding = 'unicode_escape')
df.shape

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(52870, 20)

In [11]:
# Each column's missing value counts

df.isnull().sum()

company_permalink             1
company_name                  1
company_category_code       643
company_country_code          1
company_state_code          492
company_region                1
company_city                533
investor_permalink            2
investor_name                 2
investor_category_code    50427
investor_country_code     12001
investor_state_code       16809
investor_region               2
investor_city             12480
funding_round_type            3
funded_at                     3
funded_month                  3
funded_quarter                3
funded_year                   3
raised_amount_usd          3599
dtype: int64

In [17]:
# Each column's memory footprint.
#calcualting memeory usage per chunk  
df = pd.read_csv('https://bit.ly/3BPcobU', encoding = "unicode_escape", chunksize = 5000,)

chunk_idx = 0
chunk_memory_usage = 0
initial_memory_usage = 0

for chunk in df:
  chunk_memory_usage = chunk.memory_usage(deep=True).sum()/(1024**2)
  initial_memory_usage +=chunk_memory_usage
  chunk_idx += 1
  print(f'Chunk {chunk_idx} Memory Usage: {chunk_memory_usage}MB')

print(f'\nTotal memory footprint of all chunks={initial_memory_usage}MB')


Chunk 1 Memory Usage: 5.579240798950195MB
Chunk 2 Memory Usage: 5.528232574462891MB
Chunk 3 Memory Usage: 5.535050392150879MB
Chunk 4 Memory Usage: 5.528193473815918MB
Chunk 5 Memory Usage: 5.5243377685546875MB
Chunk 6 Memory Usage: 5.553427696228027MB
Chunk 7 Memory Usage: 5.531436920166016MB
Chunk 8 Memory Usage: 5.5096588134765625MB
Chunk 9 Memory Usage: 5.396121025085449MB
Chunk 10 Memory Usage: 4.639497756958008MB
Chunk 11 Memory Usage: 2.6637144088745117MB

Total memory footprint of all chunks=56.988911628723145MB


## Question 2: Selecting Data Types

Now that we have a good sense of the missing values, let's get familiar with the column types before adding the data into SQLite.

**Tasks**

* Identify the types for each column.
* Identify the numeric columns we can represent using more space efficient types.
For text columns:
* Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.
* See if we clean clean any text columns and separate them into multiple numeric columns without adding any overhead when querying.
* Make your changes to the code from the last step so that the overall memory the data consumes stays under 10 megabytes.


In [19]:
# Identify the types for each column.
# Your code goes here
df = pd.read_csv( 'https://bit.ly/3BPcobU', encoding = "unicode_escape")
df.dtypes

company_permalink          object
company_name               object
company_category_code      object
company_country_code       object
company_state_code         object
company_region             object
company_city               object
investor_permalink         object
investor_name              object
investor_category_code     object
investor_country_code      object
investor_state_code        object
investor_region            object
investor_city              object
funding_round_type         object
funded_at                  object
funded_month               object
funded_quarter             object
funded_year               float64
raised_amount_usd         float64
dtype: object

In [20]:
# Identify the numeric columns we can represent using more space efficient types
import numpy as np

def change_to_int(df, col_name):
   
    max_col = df[col_name].max()
    min_col = df[col_name].min()
   
    for dtype_name in ['int8', 'int16', 'int32', 'int64']:
        
        if max_col <  np.iinfo(dtype_name).max and min_col > np.iinfo(dtype_name).min:
            df[col_name] = df[col_name].astype(dtype_name)
            break

use_cols = ['investor_region', 'investor_permalink', 'investor_name',
        'investor_country_code', 'investor_city', 'funding_round_type',
        'company_category_code', 'funded_at', 'company_state_code',
        'company_region', 'company_permalink', 'company_name', 
        'company_country_code', 'company_city', 'investor_state_code', 
        'funded_quarter', 'raised_amount_usd']

df = pd.read_csv( 'https://bit.ly/3BPcobU', encoding = "unicode_escape",  usecols=use_cols, chunksize=5000)
Total_memory = 0
for chunk in df:
  chunk['raised_amount_usd'] = chunk['raised_amount_usd'].fillna(0)
  change_to_int(chunk, 'raised_amount_usd' )
  chunk_memory = chunk.memory_usage(deep=True).sum() /(1024**2)
  Total_memory +=chunk_memory

  print(chunk.dtypes)
  print('==' *20)

print(f'Total Memory {Total_memory}MB')

company_permalink        object
company_name             object
company_category_code    object
company_country_code     object
company_state_code       object
company_region           object
company_city             object
investor_permalink       object
investor_name            object
investor_country_code    object
investor_state_code      object
investor_region          object
investor_city            object
funding_round_type       object
funded_at                object
funded_quarter           object
raised_amount_usd         int64
dtype: object
company_permalink        object
company_name             object
company_category_code    object
company_country_code     object
company_state_code       object
company_region           object
company_city             object
investor_permalink       object
investor_name            object
investor_country_code    object
investor_state_code      object
investor_region          object
investor_city            object
funding_round_type       o

In [22]:

#optimize string cols 
total_memory = 0
df = pd.read_csv( 'https://bit.ly/3BPcobU', encoding = "unicode_escape",  usecols=use_cols, chunksize=5000)
for chunk in df:
    # Optimize numeric column
    chunk['raised_amount_usd'] = chunk['raised_amount_usd'].fillna(0)
    change_to_int(chunk, 'raised_amount_usd' )

    # Optimize 'funded_quarter' column
    chunk['funded_quarter'] = chunk['funded_quarter'].str.extract(r'Q(\d)', expand=False)
    chunk['funded_quarter'] = chunk['funded_quarter'].fillna(-1)
    chunk['funded_quarter'] = chunk['funded_quarter'].astype('int8')

    chunk_memory = chunk.memory_usage(deep=True).sum()
    total_memory += chunk_memory
    print('Chunk memory usage (MB) =', chunk_memory/(1024**2))
print('--'*30)
print('Total memory usage (MB) =', total_memory/(1024**2))
print('--'*20)

Chunk memory usage (MB) = 4.70713996887207
Chunk memory usage (MB) = 4.827281951904297
Chunk memory usage (MB) = 4.834099769592285
Chunk memory usage (MB) = 4.827242851257324
Chunk memory usage (MB) = 4.823387145996094
Chunk memory usage (MB) = 4.871550559997559
Chunk memory usage (MB) = 4.830669403076172
Chunk memory usage (MB) = 4.808708190917969
Chunk memory usage (MB) = 4.6951704025268555
Chunk memory usage (MB) = 3.938547134399414
Chunk memory usage (MB) = 2.261368751525879
----------------------------------------
Total memory usage (MB) = 49.42516613006592
----------------------------------------


In [25]:
# Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.

df = pd.read_csv( 'https://bit.ly/3BPcobU', encoding = "unicode_escape",  usecols=use_cols, chunksize=5000)

unique_count = []
for chunk in df:
  # Optimize numeric column
  chunk['raised_amount_usd'] = chunk['raised_amount_usd'].fillna(0)
  change_to_int(chunk, 'raised_amount_usd' )


  chunk['funded_quarter'] = chunk['funded_quarter'].str.extract(r'Q(\d)', expand=False)
  chunk['funded_quarter'] = chunk['funded_quarter'].fillna(-1)
  chunk['funded_quarter'] = chunk['funded_quarter'].astype('int8')

  chunk_unique_values = chunk.select_dtypes(include=['object']).nunique() #pd.series with unique values per col
  chunk_values_count = chunk.select_dtypes(include=['object']).count() #total no of items in col
  unique_count.append((100*chunk_unique_values/chunk_values_count)) #add each chunk series to a list

total_values_uniqueness = pd.concat(unique_count)
total_values_uniqueness = total_values_uniqueness.groupby(total_values_uniqueness.index).mean()
categorise_cols = list((total_values_uniqueness[total_values_uniqueness <50]).index)
print(f'unique values percentage:\n{total_values_uniqueness}')


unique values percentage:
company_category_code     0.885323
company_city              9.884701
company_country_code      0.023168
company_name             57.691393
company_permalink        57.691393
company_region            3.899473
company_state_code        0.959790
funded_at                34.477606
funding_round_type        0.169454
investor_city             5.098421
investor_country_code     0.762508
investor_name            20.868260
investor_permalink       21.044168
investor_region           2.432371
investor_state_code       0.915164
dtype: float64


In [26]:
#optimize string cols 
# Optimize numeric column
total_memory = 0

df = pd.read_csv( 'https://bit.ly/3BPcobU', encoding = "unicode_escape",  usecols=use_cols, chunksize=5000)
for chunk in df:
   
    chunk['raised_amount_usd'] = chunk['raised_amount_usd'].fillna(0)
    change_to_int(chunk, 'raised_amount_usd' )
 
    chunk['funded_quarter'] = chunk['funded_quarter'].str.extract(r'Q(\d)', expand=False)
    chunk['funded_quarter'] = chunk['funded_quarter'].fillna(-1)
    chunk['funded_quarter'] = chunk['funded_quarter'].astype('int8')

    chunk[categorise_cols] = chunk[categorise_cols].astype('category')

    chunk_memory = chunk.memory_usage(deep=True).sum()
    total_memory += chunk_memory
    print('Chunk memory usage (MB) =', chunk_memory/(1024**2))

print('--'*20)
print('Total memory usage (MB) =', total_memory/(1024**2))

Chunk memory usage (MB) = 1.4321155548095703
Chunk memory usage (MB) = 1.253652572631836
Chunk memory usage (MB) = 1.2562618255615234
Chunk memory usage (MB) = 1.1977252960205078
Chunk memory usage (MB) = 1.2432384490966797
Chunk memory usage (MB) = 1.2561683654785156
Chunk memory usage (MB) = 1.2663860321044922
Chunk memory usage (MB) = 1.242548942565918
Chunk memory usage (MB) = 1.3025436401367188
Chunk memory usage (MB) = 1.3927536010742188
Chunk memory usage (MB) = 0.8324642181396484
----------------------------------------
Total memory usage (MB) = 13.675858497619629


## Question 3: Loading Chunks Into SQLite

Now we're in good shape to start exploring and analyzing the data. The next step is to load each chunk into a table in a SQLite database so we can query the full data set.

**Tasks**

1. Create and connect to a new SQLite database file.
2. Expand on the existing chunk processing code to export each chunk to a new table in the SQLite database.
3. Query the table and make sure the data types match up to what you had in mind for each column.

In [28]:
# Your code goes here
import sqlite3

 # Create and connect to a new database
conn = sqlite3.connect('crunchbase.db')
cur = conn.cursor()
cur.execute('''DROP TABLE IF EXISTS investments''')



df = pd.read_csv( 'https://bit.ly/3BPcobU', encoding = "latin1",  usecols=use_cols, chunksize=5000)
for chunk in df:
     
   
    chunk['raised_amount_usd'] = chunk['raised_amount_usd'].fillna(0)
    change_to_int(chunk, 'raised_amount_usd' )

    chunk = chunk[chunk['raised_amount_usd'] !=0] 

    chunk['funded_quarter'] = chunk['funded_quarter'].str.extract(r'Q(\d)', expand=False)
    chunk['funded_quarter'] = chunk['funded_quarter'].fillna(-1)
    chunk['funded_quarter'] = chunk['funded_quarter'].astype('int8')

    chunk[categorise_cols] = chunk[categorise_cols].astype('category')


    chunk.to_sql('investments', conn, if_exists='append', index=False)

In [29]:
# Query the table and make sure the data types match up to what you had in mind for each column.
df1 = pd.read_sql('PRAGMA table_info(investments)', conn)
df1

,cid,name,type,notnull,dflt_value,pk
0,0,company_permalink,TEXT,0,None,0
1,1,company_name,TEXT,0,None,0
2,2,company_category_code,TEXT,0,None,0
3,3,company_country_code,TEXT,0,None,0
4,4,company_state_code,TEXT,0,None,0
5,5,company_region,TEXT,0,None,0
6,6,company_city,TEXT,0,None,0
7,7,investor_permalink,TEXT,0,None,0
8,8,investor_name,TEXT,0,None,0
9,9,investor_country_code,TEXT,0,None,0


## Question 4: Next Steps

Now that the data is in SQLite, we can use the pandas SQLite workflow we learned in the last lesson to explore and analyze startup investments. Remember that each row isn't a unique company, but a unique investment from a single investor. This means that many startups will span multiple rows.

Use the pandas SQLite workflow to answer the following questions:

* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
* Which category of company attracted the most investments?
* Which investor contributed the most money (across all startups)?
* Which investors contributed the most money per startup?
* Which funding round was the most popular? Which was the least popular?

Here are some ideas for further exploration:

* Repeat the tasks in this project using stricter memory constraints (under 1 megabyte).
* Clean and analyze the other Crunchbase data sets from the same GitHub repo.
* Understand which columns the data sets share, and how the data sets are linked.
* Create a relational database design that links the data sets together and reduces the overall disk space the database file consumes.

Use pandas to populate each table in the database, create the appropriate indexes, and so on.

In [33]:
# Your code goes here
query = '''
    SELECT 
        company_name, 
        company_category_code,
        investor_name,
        funding_round_type,
        raised_amount_usd
    FROM 
        investments
'''
results_df = pd.read_sql(query, conn)

grouped = results_df.groupby(by='company_name').raised_amount_usd.sum()     #get results by co name and  amt
ncompanies = results_df.nunique().company_name    #get unique Co's
ordered = grouped.sort_values(ascending=False)

# What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
top_10pct = ordered.head(int(round(ncompanies*.10)))
bottom_10pct = ordered.tail(int(round(ncompanies*.10)))

print(f'\n Top 10% raised: {100*top_10pct.sum()/ grouped.sum()}%. Amount: {top_10pct.sum()} ' )
print(f'\n Bottom 10% raised: {100*bottom_10pct.sum()/ grouped.sum()}%. Amount: {bottom_10pct.sum()} ' )


 Top 10% raised: 64.56178900319603%. Amount: 440138496766 

 Bottom 10% raised: 0.02676334405486896%. Amount: 182454331 


In [34]:
# Which category of company attracted the most investments?
results_df.groupby(by='company_category_code').raised_amount_usd.sum().sort_values(ascending=False).head()

company_category_code
biotech       110396423062
software       73084516724
mobile         64777379752
cleantech      52705225028
enterprise     45860927273
Name: raised_amount_usd, dtype: int64

In [35]:
# Which investor contributed the most money (across all startups)?

results_df.groupby(by='investor_name').raised_amount_usd.sum().sort_values(ascending=False).head(1)

investor_name
Kleiner Perkins Caufield & Byers    11217826376
Name: raised_amount_usd, dtype: int64

In [36]:
# Which investors contributed the most money per startup?
sums = results_df.groupby(by=['company_name', 'investor_name']).raised_amount_usd.sum()
ranked = sums.groupby(by='company_name').rank(method='dense', ascending=False)
maxes = ranked[ranked == 1]
maxes

company_name  investor_name       
#waywire      First Round Capital     1.0
              Innovation Endeavors    1.0
              Jeff Weiner             1.0
              Oprah Winfrey           1.0
              Troy Carter             1.0
                                     ... 
ybuy          TomorrowVentures        1.0
zozi          500 Startups            1.0
              LaunchCapital           1.0
              PAR Capital Ventures    1.0
zulily        Andreessen Horowitz     1.0
Name: raised_amount_usd, Length: 24030, dtype: float64

In [37]:
# Which funding round was the most popular? Which was the least popular?
rounds = results_df.funding_round_type.value_counts()
print("Most Popular funding round: ", rounds.idxmax())
print("least popular funding round: ", rounds.idxmin())

Most Popular funding round:  series-a
least popular funding round:  crowdfunding
